In [71]:
%pip install -r requirements.txt

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 11.5 MB 585 kB/s            
ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/58/55/6fef1ef16124066b96d5b5cb107c8e0af20b2007b79ba8f7e52ca2e1b2b7/numpy-1.22.3-cp39-cp39-macosx_10_14_x86_64.whl (Caused by ReadTimeoutError("HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Read timed out. (read timeout=15)"))

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import dns.resolver
import socket
import ipaddress

outresults = {}

def execute_scan(record):
    IP_TYPES = ['AAAA', 'A']
    NAME_SERVER_TYPES = ['NS']
    ALIAS_TYPES = ['CNAME']
    ERROR_TYPES = ['NXDOMAIN']
    RECORD_TYPES = IP_TYPES + NAME_SERVER_TYPES #[aaaa, a, ns]
    NAME_TYPES = NAME_SERVER_TYPES + ALIAS_TYPES #[NS, CNAME]
    
    relationships = []
    assets = []

    has_cname = False
    
    for record_type in NAME_TYPES:
        try:
            print('looking for a', record_type, 'in', record)
            answers = dns.resolver.resolve(record, record_type)
            for answer in answers:
                if record_type in ALIAS_TYPES:
                    assets.append({ 'type' : 'DNS', 'response' : answer.to_text() })
                    print('found a', record_type)
                    has_cname = True
                    relationships.append({'source': record,'target': answer.to_text(),'relationship': record_type})
                else:
                    print(answer.to_text())
        except:
            print("No", record_type, "found")
            pass
        
    if has_cname:
        return None, relationships, assets
    
    for record_type in RECORD_TYPES:
        try:
            print('looking for a', record_type, 'in', record)
            answers = dns.resolver.resolve(record, record_type)
            for answer in answers:
                if record_type in NAME_TYPES:
                    assets.append({ 'type' : 'DNS', 'response' : answer.to_text() })
                    print('found a', record_type)
                if record_type in IP_TYPES:
                    assets.append({ 'type' : 'IP', 'response' : answer.to_text() })
                    print('found a', record_type)
                    relationships.append({'source': record,'target': answer.to_text(),'relationship': record_type})

        except:
            print("No", record_type, "found")
            pass
            
    return None, relationships, assets

In [6]:
testsubject = 'cname.branleys.com'
print(execute_scan(testsubject))  

looking for a NS in cname.branleys.com
No NS found
looking for a CNAME in cname.branleys.com
found a CNAME
(None, [{'source': 'cname.branleys.com', 'target': 'www.lloydsbank.com.', 'relationship': 'CNAME'}], [{'type': 'DNS', 'response': 'www.lloydsbank.com.'}])


In [7]:

def loopthrough(testsubject):
    result = execute_scan(testsubject)
    recordtype = result[1]
    
    for record in recordtype:
        if record['relationship'] == 'CNAME':
            print(record['source'], 'is cnamed to', record['target'])
        else:
            print("found", record['relationship'], "record for", record['source'], "-", record['target'])
            #now try to see if it exists
            makesocket(record['target'], record['source'])

def makesocket(testsubject):
    HOST = testsubject
    PORT = 80
    
    #check if its one of ours
    if isitalloydsip(HOST):
        print("its a lloyds IP no need to check for upness")
    else:
        print("attempting socket connection to", HOST, "on", PORT)
    
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.connect((HOST, PORT))
                print("connected", HOST, PORT)
            except: 
                print("couldnt connect on", PORT, "trying 443")
                PORT = 443
                try:
                    s.connect((HOST, PORT))
                    print("connected", HOST, PORT)
                except:
                    print("couldnt connect on", PORT, "possible candidate for takeover")
                return 
        #request = "GET / HTTP/1.1\r\nHost:%s\r\n\r\n" % HOST
        #s.send(request.encode())
        #data = s.recv(4096)
    
def isitalloydsip(thingip):
    addresstocheck = ipaddress.ip_address(thingip)
    lbgmainprefix = ipaddress.ip_network('141.92.0.0/16')
    
    isitoneofours = addresstocheck in lbgmainprefix
    return isitoneofours

def whatsortofrecordisit(target):
    answer = dns.resolver.resolve(target)
    rdt = dns.rdatatype.to_text(answer.rdtype)
    return rdt

In [21]:
def dns_follower(target, rtype):
    try:
        answers = dns.resolver.resolve(target, rtype)
        for rdata in answers:
            result = [target, rtype, rdata]
            return result
    except:
        result = [target, rtype, "NULL"]
        return result

In [ ]:
import pandas as pd

inputfile = open('targetlist.txt','r')
rtypes = ['CNAME', 'A', 'NS']
inlist = []
df = pd.DataFrame(inlist,
                  columns = ['targetName' , 'recordType', 'Answer'])

for target in inputfile:
    for rtype in rtypes:
        df.loc[len(df)] = dns_follower(target.strip(), rtype)
        

In [20]:
df.to_csv('outputlist.csv')

In [12]:
a_records = df['recordType'] == 'A'

df2 = df[a_records]
df2

,targetName,recordType,Answer
1,smtp1.lloydstsb.com.cn,A,67.219.247.111
3,www.clericalmedical.at,A,165.160.13.20
5,www.evolvebank.biz,A,141.92.96.30
7,www.goldfish.biz,A,37.9.175.20
9,www.lloydsbankinggroup.biz,A,141.92.90.196
11,www.lloydstsbcorporatemarkets.biz,A,141.92.96.30
13,cname.branleys.com,A,23.55.6.105


In [13]:
print(isitalloydsip('67.219.247.111'))

False


In [57]:
for answer in answers:
    print(answer)

10 aspmx.l.google.com.
20 alt1.aspmx.l.google.com.
20 alt2.aspmx.l.google.com.
30 aspmx2.googlemail.com.
30 aspmx3.googlemail.com.
